In [1]:
%pip install arabic-stopwords
%pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 9.3 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!gdown "1n6IHWly66qSJfxrDSaIBcqm05jo-v6mt"
!gdown "1zToBjksCyT4DYAZ-0Oqj52nEXk0aqtdm"
!gdown "1wKZjFyWcfsTveGpuRwXHqmfFW8Zu8cIe"
!gdown "10S8tYhh1jd5VyQD7Rbw2QVBXIfw3hd1H"
!gdown "1Fs0PTy_xPsoX5bg4QTCrX6JAaQCHRzis"
!gdown "1JLa-ELhUskQINi0syf3YJWBX8jcCXdn4"

Downloading...
From: https://drive.google.com/uc?id=1n6IHWly66qSJfxrDSaIBcqm05jo-v6mt
To: /kaggle/working/rs_train_embeddings.pkl
100%|███████████████████████████████████████| 49.5M/49.5M [00:00<00:00, 125MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zToBjksCyT4DYAZ-0Oqj52nEXk0aqtdm
To: /kaggle/working/rs_test_embeddings.pkl
100%|██████████████████████████████████████| 12.4M/12.4M [00:00<00:00, 79.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wKZjFyWcfsTveGpuRwXHqmfFW8Zu8cIe
To: /kaggle/working/ls_train_embeddings.pkl
100%|██████████████████████████████████████| 49.5M/49.5M [00:00<00:00, 77.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=10S8tYhh1jd5VyQD7Rbw2QVBXIfw3hd1H
To: /kaggle/working/ls_test_embeddings.pkl
100%|██████████████████████████████████████| 12.4M/12.4M [00:00<00:00, 93.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Fs0PTy_xPsoX5bg4QTCrX6JAaQCHRzis
To: /kaggle/working/train.pkl
100%|█████████████████████████████████████

In [3]:
import pandas as pd
import nltk
import string
import re
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.isri import ISRIStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.svm import SVC
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split, GridSearchCV
import arabicstopwords.arabicstopwords as ast
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import torch
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from collections import OrderedDict
import matplotlib.pyplot as plt
from datasets import load_dataset
from sklearn.preprocessing import OneHotEncoder

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
dataset = load_dataset('emotone_ar')

Generating train split:   0%|          | 0/10065 [00:00<?, ? examples/s]

Dataset emotone_ar downloaded and prepared to /root/.cache/huggingface/datasets/emotone_ar/default/0.0.0/36bfa7e8bdf640ccb3d5701933cfde2fa7051f22881cfab2962122771d32c430. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
def normalizeArabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    return(text)

stop_words = ast.stopwords_list()

def remove_stop_words(text):
    return ' '.join(word for word in str(text).split() if word not in stop_words)

arabic_punctuations = '''`÷×؛<>_():*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


# snow_stemmer = SnowballStemmer(language='arabic')
isri_stemmer = ISRIStemmer()

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
def stemming(text):
  x = w_tokenizer.tokenize(text)
  # return [snow_stemmer.stem(w) for w in x]
  return [isri_stemmer.stem(w) for w in x]

def prepare_text(text):
  text = normalizeArabic(text)
  text = remove_stop_words(text)
  text = remove_punctuations(text)
  text = stemming(text)
  return text

In [6]:
df = pd.DataFrame(dataset['train'])

In [7]:
df.head()

,tweet,label
0,الاوليمبياد الجايه هكون لسه ف الكليه ..,0
1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,1
2,كتنا نيله ف حظنا الهباب xD,3
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,2
4,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,0


In [8]:
df.isnull().sum()

tweet    0
label    0
dtype: int64

In [9]:
df['label'].value_counts()

0    1550
1    1444
2    1281
3    1256
4    1220
7    1207
5    1062
6    1045
Name: label, dtype: int64

In [ ]:
g = df.groupby('label')
df_balanced = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)

In [ ]:
df_balanced['original'] = df_balanced['tweet']
df_balanced['data'] = df_balanced['original'].apply(prepare_text)
df_balanced = df_balanced.drop(['tweet'], axis=1)

In [ ]:
df_balanced['label'].value_counts()

In [ ]:
df['label'].plot.hist()

In [ ]:
df_balanced['label'].plot.hist()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_balanced['data'], df_balanced['label'], test_size=0.2, stratify=df_balanced['label'], random_state=42)

In [ ]:
encoder = OneHotEncoder()

y_train = encoder.fit_transform(y_train.values.reshape(-1,1)).toarray()
y_val = encoder.transform(y_val.values.reshape(-1,1)).toarray()

In [ ]:
def prepare_tokenization(train, val, test=None, pad=500):
  tokenizer = Tokenizer(oov_token='<UNK>')
  tokenizer.fit_on_texts(train)
  tokenizer.word_index['<PAD>'] = 0
  train = tokenizer.texts_to_sequences(train)
  val = tokenizer.texts_to_sequences(val)
  if not pad==False:
    train = sequence.pad_sequences(train, maxlen=pad)
    val = sequence.pad_sequences(val, maxlen=pad)
  if not type(test)==type(None):
    test = tokenizer.texts_to_sequences(test)
    if not pad==False:
      test = sequence.pad_sequences(test, maxlen=pad)
  v_size = len(tokenizer.word_index)
  print("Vocabulary size={}".format(v_size))
  print("Number of Documents={}".format(tokenizer.document_count))
  if not type(test)==type(None):
    return v_size, train, val, test
  else:
    return v_size, train, val

In [ ]:
VOCAB_SIZE, train_sequences, val_sequences = prepare_tokenization(X_train, X_val, pad=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

train_lens = [len(s) for s in train_sequences]
plt.hist(train_lens)

In [ ]:
max_length = max(train_lens)

In [ ]:
max_length

In [ ]:
X_train = sequence.pad_sequences(train_sequences, maxlen=max_length)
X_val = sequence.pad_sequences(val_sequences, maxlen=max_length)
X_train.shape, X_val.shape

In [10]:
with open('/kaggle/working/train.pkl', 'rb') as f:
    train = pickle.load(f)

with open('/kaggle/working/test.pkl', 'rb') as f:
    test = pickle.load(f)

with open('/kaggle/working/rs_train_embeddings.pkl', 'rb') as f:
    rs_train_embeddings = pickle.load(f)
    
with open('/kaggle/working/rs_test_embeddings.pkl', 'rb') as f:
    rs_test_embeddings = pickle.load(f)
    
with open('/kaggle/working/ls_train_embeddings.pkl', 'rb') as f:
    ls_train_embeddings = pickle.load(f)
    
with open('/kaggle/working/ls_test_embeddings.pkl', 'rb') as f:
    ls_test_embeddings = pickle.load(f)

In [11]:
encoder = OneHotEncoder()

y_train = encoder.fit_transform(train['label'].values.reshape(-1,1)).toarray()
y_test = encoder.transform(test['label'].values.reshape(-1,1)).toarray()

In [12]:
train['label'].values.shape

(8052,)

In [13]:
rs_train_embeddings.shape

(8052, 768)

In [14]:
X_train_gru_global = ls_train_embeddings.reshape(ls_train_embeddings.shape[0], 1, ls_train_embeddings.shape[1]) # reshape input to allow for GRU
X_test_gru_global = ls_test_embeddings.reshape(ls_test_embeddings.shape[0], 1, ls_test_embeddings.shape[1]) # reshape input to allow for GRU

In [15]:
BATCH_SIZE = 128
EPOCHS = 50
EMBED_SIZE = 300
LEARNING_RATE =  0.001

early_stopping_monitor = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True)

In [17]:
def create_gru_model(model_type):
    model = Sequential()
    if model_type==1:
        # model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=max_length, trainable=True))
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=True)))
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=True)))
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(8, activation='softmax'))
    elif model_type==2:
        model.add(tf.keras.layers.GRU(512, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=True))
        model.add(tf.keras.layers.GRU(256, return_sequences=True))
        model.add(tf.keras.layers.GRU(128, return_sequences=False))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(8, activation='softmax'))
    elif model_type==3:
        model.add(tf.keras.layers.GRU(128, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=False))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(8, activation='softmax'))
    elif model_type==4:
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(8, activation='softmax'))
    elif model_type==5:
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(8, activation='softmax'))
    
    opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    # model.summary()

    history = model.fit(X_train_gru_global, np.asarray(y_train), validation_data=(X_test_gru_global, np.asarray(y_test)), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping_monitor])
    return model, history

In [19]:
model, _ = create_gru_model(5)
predictions = model.predict(X_test_gru_global)
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1)))

Epoch 1/50
63/63 [==============================] - 8s 44ms/step - loss: 1.5104 - accuracy: 0.4662 - val_loss: 0.9770 - val_accuracy: 0.6786
Epoch 2/50
63/63 [==============================] - 1s 24ms/step - loss: 1.0588 - accuracy: 0.6372 - val_loss: 0.8899 - val_accuracy: 0.7034
Epoch 3/50
63/63 [==============================] - 2s 24ms/step - loss: 0.9382 - accuracy: 0.6849 - val_loss: 0.8410 - val_accuracy: 0.7109
Epoch 4/50
63/63 [==============================] - 2s 24ms/step - loss: 0.8672 - accuracy: 0.7059 - val_loss: 0.8361 - val_accuracy: 0.7139
Epoch 5/50
63/63 [==============================] - 2s 25ms/step - loss: 0.8256 - accuracy: 0.7151 - val_loss: 0.8199 - val_accuracy: 0.7198
Epoch 6/50
63/63 [==============================] - 2s 24ms/step - loss: 0.7869 - accuracy: 0.7284 - val_loss: 0.8202 - val_accuracy: 0.7203
Epoch 7/50
63/63 [==============================] - 1s 24ms/step - loss: 0.7446 - accuracy: 0.7419 - val_loss: 0.8425 - val_accuracy: 0.7183
Epoch 8/50
63

In [22]:
label_to_class = {
    0: 'none',
    1: 'anger',
    2: 'joy',
    3: 'sadness',
    4: 'love',
    5: 'sympathy',
    6: 'surprise',
    7: 'fear'
}

In [20]:
predictions.shape

(2013, 8)

In [21]:
misclassified = np.where(y_test.argmax(axis=1)!=predictions.argmax(axis=1))[0]

In [23]:
def analyse_prediction(idx):
    print('Tweeet: ', test.iloc[misclassified[idx]]['tweet'])
    print('Predicted Label: ', label_to_class[np.argmax(predictions[misclassified[idx]])])
    print('True Label: ', label_to_class[test.iloc[misclassified[idx]]['label']])

In [24]:
idx = 20
analyse_prediction(idx)

Tweeet:  بانتظاركم غدا  يا احباء في مدينه  غانت الجميله  بلجيكا في ليله  نغم و  شعر Join us…
Predicted Label:  joy
True Label:  love


In [26]:
idx = 30
analyse_prediction(idx)

Tweeet:  هي لسه هتكون شكلها ! ما خلاص ياعم بقت اوسخ هي مصر راحت الاوليمبياد ليه اصلا ؟ مفيش لعبه فالحين فيها ومفشوخين ف كل الالعاب احمس
Predicted Label:  none
True Label:  sadness


In [30]:
idx = 332
analyse_prediction(idx)

Tweeet:  هو صحيح فيه لاعب من البعثه المصريه في الاوليمبياد رفع علم مصر؟ ايه السفاله دي؟
Predicted Label:  surprise
True Label:  anger


In [33]:
idx = 453
analyse_prediction(idx)

Tweeet:  ياكريم يا سخي *wink* ????❤️
Predicted Label:  sympathy
True Label:  joy


In [37]:
idx = 132
analyse_prediction(idx)

Tweeet:  09: فهد الهريفي كان احد اكبر صناع الفرح النصراوي، من كان يتوقع ان يظهر بهذا الشكل؟؟ من كان يتصور ان الاسطوره سيكون مثير للشفقه؟؟!…
Predicted Label:  anger
True Label:  sadness


In [46]:
idx = 462
analyse_prediction(idx)

Tweeet:  ليتنا ماعرفنا بعض ولا عشقنا ،! ليتني غضيت البصر ولا شفتك .. ليت العمر وقف يالغلا ولا تصادقنا ،، ليت مابه حب بهالدنيا وماحبيتًك ،!🚶🏽 💔
Predicted Label:  love
True Label:  sadness


In [47]:
idx = 100
analyse_prediction(idx)

Tweeet:  غنضت عيني وقلت برتاح.. واثره خيالك من وراء الجفن..(مرسوم)
Predicted Label:  sadness
True Label:  joy


In [55]:
idx = 137
analyse_prediction(idx)

Tweeet:  شاهد كيف يقبض الله كل شيء .. سبحانه من قابض !!    شاهد  يقبض  الله  كل  شيء  ..  سبحانه  من  قابض  !!
Predicted Label:  fear
True Label:  surprise
